First lets try to get to tridiagonal form using algorithm 26.1 from the text. This should result in tridiagonal for a hermitian matrix

In [1]:
import numpy as np
from numpy import linalg as LA

In [2]:
# following is based off from algorithm 26.1
def hessenberg(matrix):
    A=matrix.copy().astype(float)
    m = A.shape[0]

    for k in range(m-2):
        x=A[k+1:m,k]
        v = getSign(x[0])*np.linalg.norm(x,2)*getE1(x.shape[0]) + x
        v = v/np.linalg.norm(v,2)

        A[k+1:m,k:m]=A[k+1:m,k:m]-2*np.outer(v,np.matmul(v,A[k+1:m,k:m]))
        A[0:m,k+1:m]=A[0:m,k+1:m]-2*np.outer(np.matmul(A[0:m,k+1:m],v.T),v)
        
    return(A.round(8))

# function that returns an e1 vector of length m
def getE1(m):
    e1=np.zeros((m))
    e1[0]=1
    return(e1)

# a sign function that checks and corrects for x=0 according to notes in book
def getSign(x):
    sign=np.sign(x)
    if sign==0: sign=1
    return(sign)




In [3]:
A = np.array(
    [[1,2,3,4],
    [2,5,6,7],
    [3,6,8,9],
    [4,7,9,10]]
)

# A[0,0:A.shape[0]]
H=hessenberg(A)
# A.shape
print(H)

[[ 1.         -5.38516481  0.          0.        ]
 [-5.38516481 22.48275862  2.76830198  0.        ]
 [ 0.          2.76830198  0.28558894  0.17797364]
 [ 0.          0.          0.17797364  0.23165244]]


In [4]:
# check that they have the same eigenvalues
print(np.linalg.eig(A)[0])
print(np.linalg.eig(H)[0])

[24.06253512 -0.80548492  0.5580362   0.18491359]
[24.06253513 -0.80548492  0.5580362   0.18491359]


In [5]:
def fullHessenberg(matrix):
    A=matrix.copy().astype(float)
    m = A.shape[0]
    Q=np.identity(A.shape[0])

    for k in range(m-2):
        x=A[k+1:m,k]
        v = getSign(x[0])*np.linalg.norm(x,2)*getE1(x.shape[0]) + x
        v = v/np.linalg.norm(v,2)

        q_k=_getHV(v)
        P=np.identity(A.shape[0])
        P[k+1:,k+1:] = q_k
        Q=np.matmul(Q,P)

        A[k+1:m,k:m]=A[k+1:m,k:m]-2*np.outer(v,np.matmul(v,A[k+1:m,k:m]))
        A[0:m,k+1:m]=A[0:m,k+1:m]-2*np.outer(np.matmul(A[0:m,k+1:m],v.T),v)   
    return(A.round(8),Q)

def _getHV(v):
    I=np.identity(v.shape[0])
    return(I-2*np.outer(v,v))

In [6]:
H,Q = fullHessenberg(A)
print(H)
print(np.matmul(np.matmul(Q.T,A),Q).round(8))

[[ 1.         -5.38516481  0.          0.        ]
 [-5.38516481 22.48275862  2.76830198  0.        ]
 [ 0.          2.76830198  0.28558894  0.17797364]
 [ 0.          0.          0.17797364  0.23165244]]
[[ 1.         -5.38516481  0.          0.        ]
 [-5.38516481 22.48275862  2.76830198 -0.        ]
 [ 0.          2.76830198  0.28558894  0.17797364]
 [ 0.         -0.          0.17797364  0.23165244]]


In [12]:
# TRY FOR SOME S
# randomly generate some data matrix X
n = 100 # number of data points
p = 80 # number of features

X = np.random.randint(10,size=(n,p))

X_mean_array = np.mean(X,axis=0)
ones = np.ones(n)
X_mean = np.outer(ones,X_mean_array)
X_diff = X-X_mean
S = np.matmul(X_diff,X_diff.T)/n


H,Q = fullHessenberg(S)
print(H)
print(np.matmul(np.matmul(Q.T,S),Q).round(8))

[[ 7.312829   -7.93801818  0.         ... -0.          0.
  -0.        ]
 [-7.93801818 16.25048098  7.37879096 ... -0.         -0.
  -0.        ]
 [ 0.          7.37879096 12.63425302 ... -0.          0.
  -0.        ]
 ...
 [ 0.          0.         -0.         ... -0.         -0.
   0.        ]
 [ 0.          0.         -0.         ...  0.         -0.
   0.        ]
 [ 0.         -0.         -0.         ...  0.          0.
  -0.        ]]
[[ 7.312829   -7.93801818 -0.         ... -0.         -0.
   0.        ]
 [-7.93801818 16.25048098  7.37879096 ...  0.         -0.
  -0.        ]
 [-0.          7.37879096 12.63425302 ... -0.         -0.
  -0.        ]
 ...
 [-0.          0.         -0.         ...  0.          0.
   0.        ]
 [-0.         -0.         -0.         ...  0.          0.
  -0.        ]
 [ 0.         -0.         -0.         ...  0.          0.
  -0.        ]]


In [14]:
np.linalg.norm(H[-1,:],2)

0.0

In [15]:
H.shape

(100, 100)